In [1]:
import cPickle, gzip
import numpy as np
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from scipy.interpolate import spline

# Load the dataset
f = open('mnist.pkl', 'rb')
train_set, valid_set, test_set = cPickle.load(f)
f.close()

In [ ]:
train_data, train_target = train_set
test_data, test_target = test_set

Create a dataset of only zeros and ones

In [ ]:
zero_indices = train_target==0
train_zero = train_data[zero_indices]

In [ ]:
one_indices = train_target==8
train_one = train_data[one_indices]

In [ ]:
zero_indices = test_target==0
test_zero = test_data[zero_indices]

In [ ]:
one_indices = test_target==8
test_one = test_data[one_indices]

In [ ]:
my_test_labels = np.append(np.zeros(len(test_zero)), np.ones(len(test_one)))
my_train_labels = np.append(np.zeros(len(train_zero)), np.ones(len(train_one)))

my_test_data = np.concatenate((test_zero, test_one))
my_train_data = np.concatenate((train_zero, train_one))

Optional -- add in noise to make curves easier to see

In [ ]:
# will need to generate noise here

In [ ]:
#my_train_data = my_train_data*0.5 + my_noise.reshape(784)*0.5
#my_test_data = my_test_data*0.5 + my_noise.reshape(784)*0.5

Construct a neural network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
model = Sequential()
model.add(Dense(2, input_dim=784, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(2, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, init='uniform', activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='sgd')

In [ ]:
model.fit(my_train_data, my_train_labels, nb_epoch=4, batch_size=100, verbose=0)

In [ ]:
L = model.layers[0]
a = L.get_weights()[0].reshape((10, 28,28))
fig = plt.figure()
for i in range(10):
    fig.add_subplot(2, 5, i+1)
    plt.imshow(a[i], cmap='Greys_r')

In [ ]:
L.get_weights()

In [ ]:
p_zero = model.predict(test_zero)
p_one  = model.predict(test_one)

In [ ]:
plt.hist(p_zero, 50, normed=1, facecolor='blue', alpha=0.4, label='0')
plt.hist(p_one , 50, normed=1, facecolor='red' , alpha=0.4, label='1')
plt.xlabel('Smarts')
plt.ylabel('Probability')
plt.title(r'$\mathrm{Histogram\ of\ IQ:}\ \mu=100,\ \sigma=15$')
plt.yscale('log')
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

Create a ROC curve

In [ ]:
roc_points_zero = []
roc_points_one  = []

# Find min and max values of data
minval = min(p_zero.min(), p_one.min())
maxval = max(p_zero.max(), p_one.max())

for threshold in range(1001):

    # We will vary threshold from 0 to 1
    threshold = threshold/1000.0*(maxval - minval) + minval

    # Calculate proportion of 'zero' elements below the threshold
    zero_below_prop = len(np.where(p_zero < threshold)[0]) / float(len(p_zero))

    # Calculate proportion of 'one' elements above the threshold
    one_above_prop = len(np.where(p_one > threshold)[0]) / float(len(p_one))
    
    # Add this point to our ROC points
    roc_points_zero.append(zero_below_prop)
    roc_points_one.append(one_above_prop)

In [ ]:
zero_below_prop

In [ ]:
roc_points_zero

In [ ]:
plt.scatter(roc_points_zero, roc_points_one)
plt.xlim([0., 1.01])
plt.ylim([0., 1.01])
plt.show()

In [ ]:
a  = 5

In [ ]:
cars = datasets.load_cars()
X = cars.data
y = cars.target